In [1]:
save_dir = '/Users/yeonghyeon/Documents/Study/TAVE Research/데이터 분석 프로젝트팀/2. Tourism/data'

In [157]:
import time, os
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait as wait
import time

import pandas as pd
import numpy as np
import re

In [158]:
# 드라이버 지정
chrome_path = '/opt/homebrew/bin/chromedriver'
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
options.add_argument("enable-automation")
#options.add_argument('headless') # headless 모드 설정 (창 안 띄움)
options.add_argument("window-size=1920x1080") # 화면크기
options.add_argument("--no-sandbox")
options.add_argument("disable-gpu") 
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument('--disable-site-isolation-trials')


In [306]:
# 크롬 웹브라우저 실행
driver = webdriver.Chrome(chrome_path, options=options)

In [288]:
jeju_place = jeju_top47 = ['성산일출봉','한라산','만장굴','천지연','주상절리대','정방폭포','헬로키티아일랜드','제주 마라도']
len(jeju_place)


8

In [ ]:
#place_url = []

for query in jeju_place[6:]:
    driver.get('https://www.tripadvisor.co.kr/')

    # 네이버 검색창에 "검색어" 검색
    element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#lithium-root > main > div.dqRmR.dcDXR.dJjeH > div > div > div.weiIG.Z0.Wh.fRhqZ > form > input.fhEMT._G.B-.z._J.Cj.R0")))
    element.send_keys(query, Keys.ENTER)  
    time.sleep(2)

    # '즐길거리' 선택
    element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, """//*[@id="search-filters"]/ul/li[4]/a""")))
    element.click()
    time.sleep(2)

    # 관광지 검색 결과 클릭 (첫번째 글)
    if query == '제주 마라도':
        place = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 
            "#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(2) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div > div.result-title")))

    else:
        place = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 
            "#BODY_BLOCK_JQUERY_REFLOW > div.page > div > div.ui_container.main_wrap > div > div > div > div > div.content_column.ui_column.is-9-desktop.is-12-tablet.is-12-mobile > div > div.ui_columns.sections_wrapper > div > div.prw_rup.prw_search_search_results.ajax-content > div > div.main_content.ui_column.is-12 > div > div:nth-child(2) > div > div > div:nth-child(1) > div > div > div > div.ui_column.is-9-desktop.is-8-mobile.is-9-tablet.content-block-column > div > div.result-title")))
    place.click()
    time.sleep(1)
    
    # url 수집
    place_url.append(driver.current_url)


In [317]:
print(place_url)

['https://www.tripadvisor.co.kr/Attraction_Review-g297892-d1582693-Reviews-Seongsan_Ilchulbong-Seogwipo_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d550726-Reviews-Hallasan_National_Park-Seogwipo_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297885-d1643536-Reviews-Manjanggul_Cave-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d550714-Reviews-Cheonjiyeon_Falls-Seogwipo_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d1888847-Reviews-Daepo_Haean_Jusangjeolli_Cliff-Seogwipo_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d550716-Reviews-Jeongbang_Waterfall-Seogwipo_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297885-d6160806-Reviews-Hello_Kitty_Island-Jeju_Jeju_Island.html', 'https://www.tripadvisor.co.kr/Attraction_Review-g297892-d3823689-Reviews-Marado_Island-Seogwipo_Jeju_Island.html']


In [321]:
import numpy as np

def trip_advisor_crawling(url, query):
    df = pd.DataFrame(columns=['날짜', '관광지', '내용', '별점'])

    driver.get(url)

    crawl_cnt = 0 
    btn_cnt = 0
    
    actions = ActionChains(driver)  # actions 초기화 (reset_actions() 이슈)
    btn_total_cnt = int(driver.find_element_by_css_selector("#tab-data-qa-reviews-0 > div > div.dHjBB > div:nth-child(11) > div:nth-child(3) > div > div").text.split()[3])
    print('total 리뷰 개수: ', btn_total_cnt)
    
    while(True):
        # 한 페이지 안의 총 기사 개수
        time.sleep(5)
        t_len = len(driver.find_elements_by_css_selector("#tab-data-qa-reviews-0 > div > div.dHjBB > div"))
        #print('현 페이지 수: ',t_len)
                
        for i in range(1, t_len):
            # 날짜 수집
            try:
                date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "#tab-data-qa-reviews-0 > div > div.dHjBB > div:nth-child({}) > span > div > div.eRduX".format(i)))).text.split('•')[0]

            except:
                date = np.nan
                
            # 내용 수집
            text = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 
            "#tab-data-qa-reviews-0 > div > div.dHjBB > div:nth-child({}) > span > div > div.duhwe._T.bOlcm > div.pIRBV._T.KRIav > div > span".format(i)))).text
            
            # 리뷰 별점 수집
            score = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "#tab-data-qa-reviews-0 > div > div.dHjBB > div:nth-child({}) > span > div > div:nth-child(2) > svg".format(i))))
            score = float(score.get_attribute('aria-label').split()[3])
            
            df.loc[crawl_cnt] = [date, query, text, score]
            
            crawl_cnt += 1
#             if crawl_cnt % 100 == 0:
#                 print(query, crawl_cnt)
        
#             print(crawl_cnt)
        print(crawl_cnt)
        
        # 다음 버튼 누르기
        if btn_cnt != (btn_total_cnt // 10):
            try:
                btn = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, 
                                                        """//*[@id="tab-data-qa-reviews-0"]/div/div[5]/div[11]/div[1]/div/div[1]/div[2]/div/a""")))
                btn.send_keys(query, Keys.ENTER)
                btn_cnt += 1
                time.sleep(1)
                print('현재 button 개수:', btn_cnt)
            except:
                return df
            
        else:
            break
            
#         try:
#             actions.move_to_element(btn).click(btn).perform()
#         except:
#             break # 페이지 끝까지 도달해 더이상 버튼이 존재하지 않을 때

    return df

In [322]:
#trip_df = pd.DataFrame(columns=['날짜', '관광지', '내용', '별점'])

for url, query in zip(place_url[6:], jeju_place[6:]):
    tmp_df = trip_advisor_crawling(url, query)
    trip_df = pd.concat([trip_df, tmp_df]).reset_index(drop=True)

total button 개수:  56
10
현재 button 개수: 1
20
현재 button 개수: 2
30
현재 button 개수: 3
40
현재 button 개수: 4
50
현재 button 개수: 5
56
total button 개수:  56
10
현재 button 개수: 1
20
현재 button 개수: 2
30
현재 button 개수: 3
40
현재 button 개수: 4
50
현재 button 개수: 5
56


In [340]:
trip_df.head()

,날짜,관광지,내용,별점
0,2020년 12월,성산일출봉,우리나라 최남단에 제일높은산 한라산\n설경이 좋은산 사계절 어느때나 오셔도\n구경과...,5.0
1,2020년 7월,성산일출봉,산을 올라가며 바다도 볼수있어좋음!!\n중간에 가파른곳도 있지만 금방도착해서 왕복 ...,5.0
2,2020년 12월,성산일출봉,언덕이 높지 않아 올라가는데 그리 힘들지 않습니다. 산책하듯 천천히 올라가면 금방 ...,5.0
3,2020년 11월,성산일출봉,오랫만에 부부가 정상까지 올랐습니다. 제주 한 달 살이와 올레길 걷기를 계속하고 있...,5.0
4,2019년 10월,성산일출봉,느긋하게 슬렁슬렁 왔다갔다 하기 좋음.\n계단이 많아서 내려올 때 무릎이 아플 수도...,5.0


In [343]:
trip_df.to_csv(save_dir + '/trip_df.csv', index=False)